### Create DuckDB

In [1]:
import duckdb

# Create a new DuckDB database (or connect to an existing one)
conn = duckdb.connect('cleaned_redshift_data_001.duckdb')

# Define the schema and create the table
create_table_query = """
CREATE TABLE queries (
    instance_id VARCHAR,
    cluster_size INTEGER,
    user_id VARCHAR,
    database_id VARCHAR,
    query_id VARCHAR,
    arrival_timestamp TIMESTAMP,
    compile_duration_ms INTEGER,
    queue_duration_ms INTEGER,
    execution_duration_ms INTEGER,
    feature_fingerprint VARCHAR,
    was_aborted BOOLEAN,
    was_cached BOOLEAN,
    cache_source_query_id VARCHAR,
    query_type VARCHAR,
    num_permanent_tables_accessed INTEGER,
    num_external_tables_accessed INTEGER,
    num_system_tables_accessed INTEGER,
    read_table_ids VARCHAR,
    write_table_ids VARCHAR,
    mbytes_scanned INTEGER,
    mbytes_spilled INTEGER,
    num_joins INTEGER,
    num_scans INTEGER,
    num_aggregations INTEGER
);
"""

# Execute the query to create the table
conn.execute(create_table_query)

# Close the connection
conn.close()

print("DuckDB database and table created successfully.")


DuckDB database and table created successfully.


In [7]:
import duckdb

# Connection to DuckDB database
conn = duckdb.connect('cleaned_redshift_data_001.duckdb')

# Table to store the cost per query
query =  """SELECT user_id,
    DATE_TRUNC('month', arrival_timestamp) AS month,
    SUM(mbytes_scanned) AS total_data_scanned_mb,
    COUNT(query_id) AS total_queries_executed

FROM queries
GROUP BY user_id, DATE_TRUNC('month', arrival_timestamp)
ORDER BY month DESC, total_data_scanned_mb DESC
LIMIT 5;"""



# Execute the query and fetch results
result = conn.execute(query).fetchdf()  # Fetch result as a DataFrame

# Print the results
print(result)

# Close the connection
conn.close()


  user_id      month  total_data_scanned_mb  total_queries_executed
0       0 2024-05-01            702669359.0                   22336
1       2 2024-05-01              5179000.0                     165
2       1 2024-05-01              3101471.0                    2435
3       5 2024-05-01              1196376.0                      18
4       4 2024-05-01               384090.0                      39


In [8]:
import duckdb
import pandas as pd

# Connect to DuckDB database
conn = duckdb.connect('cleaned_redshift_data_001.duckdb')

# SQL query to fetch data
query = """
SELECT
    user_id,
    DATE_TRUNC('month', arrival_timestamp) AS month,
    SUM(mbytes_scanned) AS total_data_scanned_mb,
    COUNT(query_id) AS total_queries_executed
FROM
    queries
GROUP BY
    user_id, DATE_TRUNC('month', arrival_timestamp)
ORDER BY
    month DESC, total_data_scanned_mb DESC
LIMIT 5;
"""

# Execute the query and fetch results as a DataFrame
result = conn.execute(query).fetchdf()  # Fetch result as a Pandas DataFrame

# Print the results
print(result)

# Save the result as JSON
result.to_json("query_results.json", orient="records", date_format="iso")  # JSON file

# Save the result as CSV
result.to_csv("query_results.csv", index=False)  # CSV file

# Close the connection
conn.close()

print("Query results saved as 'query_results.json' and 'query_results.csv'.")


  user_id      month  total_data_scanned_mb  total_queries_executed
0       0 2024-05-01            702669359.0                   22336
1       2 2024-05-01              5179000.0                     165
2       1 2024-05-01              3101471.0                    2435
3       5 2024-05-01              1196376.0                      18
4       4 2024-05-01               384090.0                      39
Query results saved as 'query_results.json' and 'query_results.csv'.


In [10]:
import duckdb

# Connect to DuckDB database
conn = duckdb.connect('cleaned_redshift_data_001.duckdb')

query = """select query_id , instance_id from queries where query_id = 143296;"""

# Execute the query and fetch results
result = conn.execute(query).fetchdf()  # Fetch result as a DataFrame

# Print the results
print(result)



# Query 2: Create the `cost_estimation` table
query2 = """
CREATE TABLE IF NOT EXISTS cost_estimation (
    query_id  VARCHAR PRIMARY KEY,
    user_id VARCHAR,
    cluster_size INTEGER,
    arrival_timestamp TIMESTAMP,
    execution_duration_ms INTEGER,
    mbytes_scanned INTEGER,
    mbytes_spilled INTEGER,
    total_execution_cost DECIMAL(10, 4),
    total_scan_cost DECIMAL(10, 4),
    total_spill_cost DECIMAL(10, 4),
    total_cost DECIMAL(10, 4)
);
"""
conn.execute(query2)

# Query 3: Insert the cost data into `cost_estimation` (assume cost calculations are done separately)
query3 = """
INSERT INTO cost_estimation (
    query_id, user_id, cluster_size, arrival_timestamp,
    execution_duration_ms, mbytes_scanned, mbytes_spilled,
    total_execution_cost, total_scan_cost, total_spill_cost, total_cost
)
SELECT
    q.query_id ||'_'||q.instance_id,
    q.user_id,
    q.cluster_size,
    q.arrival_timestamp,
    q.execution_duration_ms,
    q.mbytes_scanned,
    q.mbytes_spilled,
    ROUND(q.execution_duration_ms / 1000 * 0.000138, 4) AS total_execution_cost,
    ROUND(q.mbytes_scanned / 1024 * 0.03, 4) AS total_scan_cost,
    ROUND(q.mbytes_spilled / 1024 * 0.02, 4) AS total_spill_cost,
    ROUND(
        ROUND(q.execution_duration_ms / 1000 * 0.000138, 4) +
        ROUND(q.mbytes_scanned / 1024 * 0.03, 4) +
        ROUND(q.mbytes_spilled / 1024 * 0.02, 4),
        4
    ) AS total_cost
FROM queries q;
"""


result = conn.execute(query3).fetchdf()  # Fetch result as a DataFrame

# Print the results
print(result)

# Close the connection
conn.close()


  query_id instance_id
0   143296         104
   Count
0  82444


In [21]:
import duckdb

# Connect to DuckDB database
conn = duckdb.connect('cleaned_redshift_data_001.duckdb')

query = """select * from queries where query_id = 143296;"""

# Execute the query and fetch results
result = conn.execute(query).fetchdf()  # Fetch result as a DataFrame

# Print the results
print(result)

# Close the connection


  instance_id  cluster_size user_id database_id query_id  \
0         104           NaN       0           0   143296   

           arrival_timestamp  compile_duration_ms  queue_duration_ms  \
0 2024-03-01 00:02:17.633593               760245                  0   

   execution_duration_ms                                feature_fingerprint  \
0                 780366  f93d7a158afa58c0981874d43045b0978c5da8a0d7accf...   

   ...  num_permanent_tables_accessed  num_external_tables_accessed  \
0  ...                              2                             0   

  num_system_tables_accessed read_table_ids  write_table_ids  mbytes_scanned  \
0                          0              6             None          124716   

   mbytes_spilled num_joins num_scans  num_aggregations  
0               0         1         2                 3  

[1 rows x 24 columns]


In [29]:
import duckdb
import pandas as pd

conn = duckdb.connect('cleaned_redshift_data_001.duckdb')
# SQL Query to identify tables used only in JOINs
query = """
WITH exploded_tables AS (
    SELECT
        query_id,
        num_joins,
        UNNEST(SPLIT(read_table_ids, ',')) AS table_id
    FROM
        queries
    WHERE
        read_table_ids IS NOT NULL
),
join_usage AS (
    SELECT
        table_id,
        COUNT(*) AS join_count
    FROM
        exploded_tables
    WHERE
        num_joins > 0
    GROUP BY
        table_id
),
total_usage AS (
    SELECT
        table_id,
        COUNT(*) AS total_count
    FROM
        exploded_tables
    GROUP BY
        table_id
),
usage_analysis AS (
    SELECT
        t.table_id,
        t.total_count,
        j.join_count,
        (j.join_count * 1.0 / t.total_count) AS join_usage_ratio
    FROM
        total_usage t
    LEFT JOIN
        join_usage j
    ON
        t.table_id = j.table_id
)
SELECT
    table_id,
    total_count,
    join_count,
    join_usage_ratio
FROM
    usage_analysis
WHERE
    join_usage_ratio = 1.0
ORDER BY
    total_count DESC;
"""

# Execute the query and fetch the results
result = conn.execute(query).fetchdf()

# Save the result as a JSON file
output_json_file = '/tables_only_used_in_joins.json'
result.to_json(output_json_file, orient="records")

# Print the results and file path
print("Query executed successfully!")
print(f"Results saved to: {output_json_file}")



Query executed successfully!
Results saved to: /tables_only_used_in_joins.json


In [30]:
import duckdb
import pandas as pd



# SQL Query to calculate execution time per query type
query = """
SELECT
    query_type,
    COUNT(*) AS query_count,
    SUM(execution_duration_ms) AS total_execution_time_ms,
    AVG(execution_duration_ms) AS average_execution_time_ms
FROM
    queries
GROUP BY
    query_type
ORDER BY
    total_execution_time_ms DESC;
"""

# Execute the query and fetch the results
result = conn.execute(query).fetchdf()
print(result)
# Save the result as a JSON file
output_json_file = '/query_type_execution_time.json'
result.to_json(output_json_file, orient="records")

# Print the results and file path
print("Query executed successfully!")
print(f"Results saved to: {output_json_file}")

# Close the connection
conn.close()


  query_type  query_count  total_execution_time_ms  average_execution_time_ms
0     select        21647              318336220.0               14705.789255
1     insert        29754              223870189.0                7524.036735
2     delete         7433              174988219.0               23542.071707
3       copy        10543              121621747.0               11535.781751
4     update          200               35616582.0              178082.910000
5       ctas         1281               32901215.0               25684.008587
6      other         6261               22507397.0                3594.856572
7     vacuum          155               21001070.0              135490.774194
8    analyze         3445                6369264.0                1848.842961
9     unload         1725                3776547.0                2189.302609
Query executed successfully!
Results saved to: /query_type_execution_time.json
